<div class="alert alert-block alert-info">
<b>HSN-TABLE Wise GST Summary Report:</b> 
</div>

*  Make Pivot Summery of Sale data for GSTR-1 reporting.
- Report:
1. HSN Wise Summery
2. Table Wise Summery (With GST Rate)
   - Table 4A- B2B
   - Table 5A- B2C
   - Table 9- CDNR
3. Main Data With all above workings
            

In [1]:
import pandas as pd
import numpy as np

In [7]:
# Extract Data
cust_master = pd.read_excel(input("Import Customer Master"))
prod_master = pd.read_excel(input("Import Product Master"))
state_code_master = pd.read_excel(input("Import state code master Master"))
main_sales_data = pd.read_excel(input("Import Sales Master"),skiprows=2)

### Check Data Quality

In [ ]:
main_sales_data.head(2)

,Supplier State,GST Number,Product Code,Doc Date,Doc Type,Units Sold,Discount
0,BIHAR,28ETWDV2926V2Y1,1004,2020-01-01,Invoice,1618.5,80.925
1,BIHAR,19TTWXT2113D2S2,1001,2020-01-01,Invoice,1321.0,66.050


In [9]:
cust_master.head(2)

,GST NUMBER OF CUSTOMER,NAME OF CUSTOMER,EMAIL ID,PHONE NUMBER,GSTN STATUS
0,10DQHBZ9724D9O3,PRABHA ELECTRONICS PVT. LTD.,NaN,NaN,ACT
1,10EBWDE4738G5A6,UNITY CYLINDERS & EQUIPMENTS PRIVATE LIMITED,NaN,NaN,ACT


In [10]:
prod_master.head(2)

,S.N,Product Code,Product Code2,HSN Code,GST RATE,PRICE
0,1,1001,Oil,5542,0.12,45
1,2,1004,Beverage,8462,0.28,80


In [11]:
state_code_master.head(2)

,State Code,State Name
0,1,JAMMU AND KASHMIR
1,2,HIMACHAL PRADESH


In [15]:
#state_code_master.info()
#cust_master.info()
#prod_master.info()
main_sales_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2500 entries, 0 to 2499
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   Supplier State  2500 non-null   object        
 1   GST Number      2464 non-null   object        
 2   Product Code    2500 non-null   int64         
 3   Doc Date        2500 non-null   datetime64[ns]
 4   Doc Type        2500 non-null   object        
 5   Units Sold      2500 non-null   float64       
 6   Discount        2500 non-null   float64       
dtypes: datetime64[ns](1), float64(2), int64(1), object(3)
memory usage: 136.8+ KB


### Transform Data

In [ ]:
main_template = main_sales_data
# Get Product Details
df2=main_template.merge(right=prod_master,how="left",left_on="Product Code",right_on="Product Code")
df2.drop('S.N',axis=1,inplace=True)

In [20]:
df2.head(2)

,Supplier State,GST Number,Product Code,Doc Date,Doc Type,Units Sold,Discount,Product Code2,HSN Code,GST RATE,PRICE
0,BIHAR,28ETWDV2926V2Y1,1004,2020-01-01,Invoice,1618.5,80.925,Beverage,8462,0.28,80
1,BIHAR,19TTWXT2113D2S2,1001,2020-01-01,Invoice,1321.0,66.050,Oil,5542,0.12,45


In [21]:
# Find Taxable Value
df2['Basic Sale']=df2['Units Sold']*df2['PRICE']
df2['Taxable Value']=df2['Basic Sale']-df2['Discount']

In [22]:
df2.head(2)

,Supplier State,GST Number,Product Code,Doc Date,Doc Type,Units Sold,Discount,Product Code2,HSN Code,GST RATE,PRICE,Basic Sale,Taxable Value
0,BIHAR,28ETWDV2926V2Y1,1004,2020-01-01,Invoice,1618.5,80.925,Beverage,8462,0.28,80,129480.0,129399.075
1,BIHAR,19TTWXT2113D2S2,1001,2020-01-01,Invoice,1321.0,66.050,Oil,5542,0.12,45,59445.0,59378.950



#### Get State information for IGST C-SGST Bifurcation: -

- Get State-Code from GST Number
- merge State_Master with main_data with state code
- Get CGST/SGST , IGST from State info.

In [23]:
def get_state_code(gst_number):
    try:
        return int(gst_number[0:2])
    except:
        return 0


df2['State Code']=df2['GST Number'].apply(lambda x: get_state_code(x))
df3=df2.merge(right=state_code_master,how='left',left_on='State Code',right_on='State Code')
df3.drop('State Code', axis=1, inplace=True)

In [24]:
df3.head(2)

,Supplier State,GST Number,Product Code,Doc Date,Doc Type,Units Sold,Discount,Product Code2,HSN Code,GST RATE,PRICE,Basic Sale,Taxable Value,State Name
0,BIHAR,28ETWDV2926V2Y1,1004,2020-01-01,Invoice,1618.5,80.925,Beverage,8462,0.28,80,129480.0,129399.075,ANDHRA PRADESH(BEFORE DIVISION)
1,BIHAR,19TTWXT2113D2S2,1001,2020-01-01,Invoice,1321.0,66.050,Oil,5542,0.12,45,59445.0,59378.950,WEST BENGAL


In [25]:
df3['IGST']=np.where(df3['Supplier State']==df3['State Name'],0,df3['Taxable Value']*df3['GST RATE'])
df3['CGST']=np.where(df3['Supplier State']!=df3['State Name'],0,df3['Taxable Value']*df3['GST RATE']/2)
df3['SGST']=np.where(df3['Supplier State']!=df3['State Name'],0,df3['Taxable Value']*df3['GST RATE']/2)
df3['Total GST']=df3['IGST']+df3['CGST']+df3['SGST']

In [29]:
# Check IGST 
df3[df3['IGST']>0].head(2)

,Supplier State,GST Number,Product Code,Doc Date,Doc Type,Units Sold,Discount,Product Code2,HSN Code,GST RATE,PRICE,Basic Sale,Taxable Value,State Name,IGST,CGST,SGST,Total GST
0,BIHAR,28ETWDV2926V2Y1,1004,2020-01-01,Invoice,1618.5,80.925,Beverage,8462,0.28,80,129480.0,129399.075,ANDHRA PRADESH(BEFORE DIVISION),36231.741,0.0,0.0,36231.741
1,BIHAR,19TTWXT2113D2S2,1001,2020-01-01,Invoice,1321.0,66.050,Oil,5542,0.12,45,59445.0,59378.950,WEST BENGAL,7125.474,0.0,0.0,7125.474


In [30]:
# Check CGST/SGST
df3[df3['CGST']>0].head(2)

,Supplier State,GST Number,Product Code,Doc Date,Doc Type,Units Sold,Discount,Product Code2,HSN Code,GST RATE,PRICE,Basic Sale,Taxable Value,State Name,IGST,CGST,SGST,Total GST
4,BIHAR,10DQHBZ9724D9O3,1310,2020-06-06,Invoice,2470.0,123.50,Shampoo,5632,0.12,140,345800.0,345676.50,BIHAR,0.0,20740.590,20740.590,41481.18
9,BIHAR,10EBWDE4738G5A6,1310,2020-06-06,Invoice,1545.0,77.25,Shampoo,5632,0.12,140,216300.0,216222.75,BIHAR,0.0,12973.365,12973.365,25946.73


In [36]:
# Replace Nan Values
df3 = df3.replace(np.nan, "", regex=True)

### Bifurcate B2B / B2C / CDNR

In [37]:
conditions=[((df3['Doc Type']=='Invoice')&(df3['GST Number']!="")),
            ((df3['Doc Type']=='Invoice')&(df3['GST Number']=="")),
            (df3['Doc Type']!='Invoice')]
results=["Table 4A - B2B","Table 5A - B2C","Table 9 - CDNR"]

df3['GSTR-1 Table']=np.select(conditions,results)


In [47]:
# Check B2B/B2C/CDNR
#df3[df3['GSTR-1 Table']=="Table 5A - B2C"].head(2)
#df3[df3['GSTR-1 Table']=="Table 4A - B2B"].head(2)
df3[df3['GSTR-1 Table']=="Table 9 - CDNR"].head(2)

,Supplier State,GST Number,Product Code,Doc Date,Doc Type,Units Sold,Discount,Product Code2,HSN Code,GST RATE,PRICE,Basic Sale,Taxable Value,State Name,IGST,CGST,SGST,Total GST,GSTR-1 Table
88,BIHAR,23CEECB7723Z0R6,1008,2020-03-03,Credit Note,263.0,13.15,Soap,4975,0.12,90,23670.0,23656.85,MADHYA PRADESH,2838.822,0.0,0.0,2838.822,Table 9 - CDNR
99,BIHAR,12TJJSJ1298B2I6,1210,2020-10-10,Credit Note,214.0,10.70,Juice,5524,0.12,120,25680.0,25669.30,ARUNACHAL PRADESH,3080.316,0.0,0.0,3080.316,Table 9 - CDNR


<div class="alert alert-block alert-info">
<b>HSN Wise Summery:</b> 
</div>

In [66]:
HSN_sum=df3.groupby(by='HSN Code',as_index=False)[['CGST','SGST','IGST','Total GST']].sum()
HSN_sum

,HSN Code,CGST,SGST,IGST,Total GST
0,4975,387806.832,387806.832,8.193493e+06,8.969107e+06
1,5524,422881.326,422881.326,1.191184e+07,1.275760e+07
2,5542,128263.926,128263.926,3.981538e+06,4.238066e+06
3,5632,595817.532,595817.532,1.161883e+07,1.281046e+07
4,5636,0.000,0.000,2.781625e+06,2.781625e+06
5,8462,300308.190,300308.190,1.513297e+07,1.573359e+07


<div class="alert alert-block alert-info">
<b>GSTR-1 TABLE Wise Summary :</b> 
</div>

In [71]:
Table_sum=df3.pivot_table(values=['CGST','SGST','IGST','Total GST'],index=['GSTR-1 Table'],aggfunc='sum').reset_index()
Table_sum

,GSTR-1 Table,CGST,IGST,SGST,Total GST
0,Table 4A - B2B,1826603.742,5.272139e+07,1826603.742,5.637460e+07
1,Table 5A - B2C,0.000,7.163396e+05,0.000,7.163396e+05
2,Table 9 - CDNR,8474.064,1.825637e+05,8474.064,1.995119e+05


In [72]:
# With GST Rate
Table_Rate_sum=df3.pivot_table(values=['CGST','SGST','IGST','Total GST'],index=['GSTR-1 Table','GST RATE'],aggfunc='sum')
Table_Rate_sum

CGST          IGST         SGST     Total GST
GSTR-1 Table   GST RATE                                                      
Table 4A - B2B 0.12      1531175.700  3.505524e+07  1531175.700  3.811759e+07
               0.18            0.000  2.781625e+06        0.000  2.781625e+06
               0.28       295428.042  1.488453e+07   295428.042  1.547539e+07
Table 5A - B2C 0.12            0.000  5.145970e+05        0.000  5.145970e+05
               0.28            0.000  2.017426e+05        0.000  2.017426e+05
Table 9 - CDNR 0.12         3593.916  1.358665e+05     3593.916  1.430544e+05
               0.28         4880.148  4.669720e+04     4880.148  5.645749e+04

<div class="alert alert-block alert-warning">
<b>Export all summery reports To Excel:</b> 
</div>

In [ ]:
writer=pd.ExcelWriter(f"{input("Store location")}\{input("File Name")}.xlsx",engine='openpyxl')
HSN_sum.to_excel(writer,sheet_name="HSN Wise",index=False)
Table_sum.to_excel(writer,sheet_name="R1 Table",index=False)
Table_Rate_sum.to_excel(writer,sheet_name="R1 Table & Rate")
df3.to_excel(writer,sheet_name="Main Data",index=False)
writer.close()